In [33]:
import contractions
import pandas as pd
import numpy as np
import re
import emoji
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     /home/copper_turtle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/copper_turtle/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/copper_turtle/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
df = pd.read_csv("tweet_data.csv")

In [35]:

def replace_retweet(tweet, default_replace=""):
  tweet = re.sub("^RT\s+", default_replace, tweet)
  return tweet

def replace_user(tweet, default_replace="twitteruser"):
  tweet = re.sub("\B@\w+", default_replace, tweet)
  return tweet

def replace_url(tweet, default_replace=""):
  tweet = re.sub('(http|https):\/\/\S+', default_replace, tweet)
  return tweet

def replace_hashtag(tweet, default_replace=""):
  tweet = re.sub('#+', default_replace, tweet)
  return tweet

def to_lowercase(tweet):
  return tweet.lower()

def word_repetition(tweet):
  tweet = re.sub(r'(.)\1+', r'\1\1', tweet)
  return tweet

def punct_repetition(tweet, default_replace=""):
  tweet = re.sub(r'[\?\.\!]+(?=[\?\.!])', default_replace, tweet)
  return tweet

def fix_contractions(tweet):
  return contractions.fix(tweet)

def tokenize(tweet):
  tokens = word_tokenize(tweet)
  return tokens

def demojize(tweet):
    tweet = emoji.demojize(tweet)
    return tweet



<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\B'
<>:10: SyntaxWarning: invalid escape sequence '\/'
<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\B'
<>:10: SyntaxWarning: invalid escape sequence '\/'
/tmp/ipykernel_71430/2047234332.py:2: SyntaxWarning: invalid escape sequence '\s'
  tweet = re.sub("^RT\s+", default_replace, tweet)
/tmp/ipykernel_71430/2047234332.py:6: SyntaxWarning: invalid escape sequence '\B'
  tweet = re.sub("\B@\w+", default_replace, tweet)
/tmp/ipykernel_71430/2047234332.py:10: SyntaxWarning: invalid escape sequence '\/'
  tweet = re.sub('(http|https):\/\/\S+', default_replace, tweet)


In [36]:
def custom_tokenize(tweet, keep_punct = False, keep_alnum = False, keep_stop = False):
    token_list = word_tokenize(tweet)
    
    if not keep_punct:
        token_list = [token for token in token_list if token not in string.punctuation]
        
    if not keep_alnum:
        token_list = [token for token in token_list if token.isalpha()]
    
    if not keep_stop:
        stop_words = set(stopwords.words('english'))
        stop_words.discard('not')
        token_list = [token for token in token_list if not token in stop_words]
        
          
    
    return token_list

In [37]:
porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer()
snowball_stemmer = SnowballStemmer('english')

In [38]:
def stem_tokens(tokens, stemmer):
    token_list = []
    for token in tokens:
        token_list.append(stemmer.stem(token))
    return token_list

In [39]:
def process_tweet(tweet, verbose=False):
  if verbose: print("Initial tweet: {}".format(tweet))
  print(tweet)



  ## Twitter Features
    # replace retweet
  tweet = replace_retweet(tweet, default_replace="")
  # replace user tag
  tweet = replace_user(tweet, default_replace="twitteruser")
  # replace url
  tweet = replace_url(tweet, default_replace="")
  # replace hashtag
  tweet = replace_hashtag(tweet, default_replace="")
  if verbose: print("Post Twitter processing tweet: {}".format(tweet))

  ## Word Features
    # lower case
  tweet = to_lowercase(tweet)
    # replace contractions
  tweet = fix_contractions(tweet)
    # replace punctuation repetition
  tweet = punct_repetition(tweet, default_replace="")
    # replace word repetition
  tweet = word_repetition(tweet)
    # replace emojis
  tweet = demojize(tweet)
  if verbose: print("Post Word processing tweet: {}".format(tweet))

  ## Tokenization & Stemming
    # tokenize
  tokens = custom_tokenize(tweet, keep_alnum=True)
    # define stemmer
    # stem tokens
  stem = stem_tokens(tokens, SnowballStemmer("english"))
  return stem

In [69]:
stop_words = set(stopwords.words('english'))
print(stop_words)
df

{'what', 'isn', 'been', 'at', 'myself', 'you', 'in', 'down', "weren't", 'doing', 'o', 'any', 'didn', 'but', 's', 'their', 'about', 'should', 'was', 'a', 'here', 'some', 'did', 'theirs', 'or', "that'll", 'other', "hasn't", 'were', 'up', 'why', "won't", 'aren', 'shouldn', 'and', 'don', 'an', 'yours', 'do', 'hers', 'ain', 'be', 'once', 'we', 'while', "needn't", 'ours', 'himself', 'against', 'is', 'can', 'themselves', 'whom', 'how', 'there', 'me', 'him', 'such', 'haven', 'where', 'couldn', 'only', 'herself', "you'd", 'are', 'he', 'of', 'her', 'by', 'so', 'mustn', 'my', 'own', 're', 'won', 'few', 'this', "aren't", 'no', "wasn't", 'when', 'to', 'mightn', 't', 'being', 'until', 'both', 'ourselves', "she's", "didn't", 'y', 'more', 'ma', 'them', "shan't", 'hadn', "don't", 'it', 'have', "couldn't", 'has', 'over', 'above', 'under', 'its', 'the', 'each', 'before', 'll', 'wasn', 'for', 'if', 'as', "mustn't", 'than', "should've", 'too', 'further', 'off', 'will', "you'll", 'after', 'nor', 'had', "you

,textID,tweet_text,sentiment,tokens,tweet_sentiment
0,1956967666,Layin n bed with a headache ughhhh...waitin o...,negative,"[layin, n, bed, headach, ughh.waitin, call]",0
1,1956967696,Funeral ceremony...gloomy friday...,negative,"[funer, ceremony.gloomi, friday]",0
2,1956967789,wants to hang out with friends SOON!,positive,"[want, hang, friend, soon]",1
3,1956968477,Re-pinging @ghostridah14: why didn't you go to...,negative,"[re-ping, twitterus, not, go, prom, bf, not, l...",0
4,1956968636,Hmmm. http://www.djhero.com/ is down,negative,[hmm],0
...,...,...,...,...,...
18722,1753918818,"had SUCH and AMAZING time last night, McFly we...",positive,"[amaz, time, last, night, mcfli, incred]",1
18723,1753918881,@jasimmo Ooo showing of your French skills!! l...,positive,"[twitterus, oo, show, french, skill, lol, thin...",1
18724,1753918900,Succesfully following Tayla!!,positive,"[succes, follow, tayla]",1
18725,1753919001,Happy Mothers Day All my love,positive,"[happi, mother, day, love]",1


In [67]:
df["tokens"] = df["tweet_text"].apply(process_tweet)
df["tweet_sentiment"] = df["sentiment"].apply(lambda i: 1
                                              if i == 'positive' else 0)




Layin n bed with a headache  ughhhh...waitin on your call...
Funeral ceremony...gloomy friday...
wants to hang out with friends SOON!
Re-pinging @ghostridah14: why didn't you go to prom? BC my bf didn't like my friends
Hmmm. http://www.djhero.com/ is down
@charviray Charlene my love. I miss you
@kelcouch I'm sorry  at least it's Friday?
Choked on her retainers
Ugh! I have to beat this stupid song to get to the next  rude!
@BrodyJenner if u watch the hills in london u will realise what tourture it is because were weeks and weeks late  i just watch itonlinelol
So sleepy again and it's not even that late. I fail once again.
@PerezHilton lady gaga tweeted about not being impressed by her video leaking just so you know
How are YOU convinced that I have always wanted you? What signals did I give off...damn I think I just lost another friend
On my way home n having 2 deal w underage girls drinking gin on da bus while talking bout keggers......damn i feel old
@IsaacMascote  i'm sorry people ar

(18727,)

In [68]:
df["tweet_sentiment"]

0        0
1        0
2        1
3        0
4        0
        ..
18722    1
18723    1
18724    1
18725    1
18726    1
Name: tweet_sentiment, Length: 18727, dtype: int64

In [65]:
X = df["tokens"].tolist()
y = df["tweet_sentiment"].tolist()

[['layin', 'n', 'bed', 'headach', 'ughh.waitin', 'call'],
 ['funer', 'ceremony.gloomi', 'friday'],
 ['want', 'hang', 'friend', 'soon'],
 ['re-ping', 'twitterus', 'not', 'go', 'prom', 'bf', 'not', 'like', 'friend'],
 ['hmm'],
 ['twitterus', 'charlen', 'love', 'miss'],
 ['twitterus', 'sorri', 'least', 'friday'],
 ['choke', 'retain'],
 ['ugh', 'beat', 'stupid', 'song', 'get', 'next', 'rude'],
 ['twitterus',
  'watch',
  'hill',
  'london',
  'realis',
  'tourtur',
  'week',
  'week',
  'late',
  'watch',
  'itonlinelol'],
 ['sleepi', 'not', 'even', 'late', 'fail'],
 ['twitterus',
  'ladi',
  'gaga',
  'tweet',
  'not',
  'impress',
  'video',
  'leak',
  'know'],
 ['convinc',
  'alway',
  'want',
  'signal',
  'give',
  'off.damn',
  'think',
  'lost',
  'anoth',
  'friend'],
 ['way',
  'home',
  'n',
  '2',
  'deal',
  'w',
  'underag',
  'girl',
  'drink',
  'gin',
  'da',
  'bus',
  'talk',
  'bout',
  'keggers.damn',
  'feel',
  'old'],
 ['twitterus',
  'sorri',
  'peopl',
  'rude',
 

In [43]:
def fit_cv(tweet_corpus):
    cv_vect = CountVectorizer(tokenizer=lambda x: x,
                              preprocessor=lambda x: x)
    cv_vect.fit(tweet_corpus)
    return  cv_vect

In [44]:
def fit_tfid(tweet_corpus):
    tf_vect = TfidfVectorizer(preprocessor=lambda x: x,
                              tokenizer=lambda x: x)
    tf_vect.fit(tweet_corpus)
    return tf_vect

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=0.80)

In [46]:
print("Size of X_train: {}".format(len(X_train)))
print("Size of y_train: {}".format(len(y_train)))
print("\n")
print("Size of X_test: {}".format(len(X_test)))
print("Size of y_test: {}".format(len(y_test)))
print("\n")
print("Train proportion: {:.0%}".format(len(X_train)/
                                        (len(X_train)+len(X_test))))

Size of X_train: 14981
Size of y_train: 14981


Size of X_test: 3746
Size of y_test: 3746


Train proportion: 80%


In [47]:
cv = fit_cv(X_train)
X_train_cv = cv.transform(X_train)
X_test_cv = cv.transform(X_test)

/home/copper_turtle/anaconda3/envs/ai_ml/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [48]:
def fit_lr(X_train, y_train):
    model = LogisticRegression()
    model.fit(X_train, y_train)
    return model

In [49]:
model_lr_cv = fit_lr(X_train_cv, y_train)

In [50]:
tf = fit_tfid(X_train)
X_train_tf = tf.transform(X_train)
X_test_tf = tf.transform(X_test)

/home/copper_turtle/anaconda3/envs/ai_ml/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [51]:
model_lr_tf = fit_lr(X_train_tf, y_train)

In [52]:
cv = fit_cv(X_train)
X_train_cv = cv.transform(X_train)
X_test_cv = cv.transform(X_test)

/home/copper_turtle/anaconda3/envs/ai_ml/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [53]:
model_lr_cv = fit_lr(X_train_cv, y_train)

In [54]:
model_lr_tf = fit_lr(X_train_tf, y_train)

In [55]:
y_pred_lr_cv = model_lr_cv.predict(X_test_cv)

In [56]:
print("CV LR Model Accuracy: {:.2%}".format(accuracy_score(y_test, y_pred_lr_cv)))

CV LR Model Accuracy: 88.87%


In [57]:
y_pred_lr_tf = model_lr_tf.predict(X_test_tf)

In [58]:
print("TFID LR Model Accuracy: {:.2%}".format(accuracy_score(y_test, y_pred_lr_tf)))

TFID LR Model Accuracy: 88.55%


In [59]:
def predict_tweet(tweet):
    processed_tweet = process_tweet(tweet)
    transformed_tweet = tf.transform([processed_tweet])
    prediction = model_lr_tf.predict(transformed_tweet)
    
    if prediction == 1:
        return "Prediction pos"
    else:
        return "Predict negative"

In [60]:
final = predict_tweet("@PerezHilton lady gaga tweeted about not being impressed by her video leaking just so you know")

print(f"Final: {final}")

@PerezHilton lady gaga tweeted about not being impressed by her video leaking just so you know
Final: Predict negative


In [61]:
from sklearn.svm import SVC
from sklearn import metrics

In [62]:
svc_model = SVC(gamma='auto')
svc_model.fit(X_train, y_train)
predictions =svc_model.predict(X_test)
print(metrics.confusion_matrix(y_test, predictions))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (14981,) + inhomogeneous part.